<a href="https://colab.research.google.com/github/tgalbaugh/pythonProject/blob/master/pythonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount, import etc

Select CSV file

In [14]:
import pandas as pd
import io
import numpy as np
import datetime as dt
import pytz as tz
import sys

In [31]:
#df = pd.read_csv(io.BytesIO(['file.csv']))
#print(df)
#%cd '//content/pythonProject/sourceCSV'
#%cd '//content/pythonProject'
df = pd.read_csv ('//content/pythonProject/sourceCSV/warehousedata-prodsheet-20220101.csv')
print(df)
#!dir


      99  44562  AUTOPROCESS  161     1600  0  \
0     99  44562  AUTOPROCESS  161     1600  0   
1     99  44562  AUTOPROCESS  161     1600  0   
2     99  44562  AUTOPROCESS  161     1600  0   
3     99  44562  AUTOPROCESS  162     1600  0   
4     99  44562  AUTOPROCESS  162     1600  0   
...   ..    ...          ...  ...      ... ..   
3524  99  44562      4047454  161     1610  0   
3525  99  44562      4047454  161     5600  0   
3526  99  44562      4047454  161     1610  0   
3527  99  44562      4047454  161     5600  0   
3528  99  44562      4047454  161  9997161  0   

      1    NULNULNULL  NULL  NULL NUL  231    0.1  2.715  ...  Unnamed: 73  \
0     1    NULNULNULL  NULL  NULL NUL  474  0.000  5.572  ...          NaN   
1     1    NULNULNULL  NULL  NULL NUL  117  0.000  1.375  ...          NaN   
2     1    NULNULNULL  NULL  NULL NUL  141  0.000  1.657  ...          NaN   
3     1    NULNULNULL  NULL  NULL NUL  293  0.000  3.444  ...          NaN   
4     1    NULNULNULL

In [39]:
#request = drive_service.files().get_media(fileId=fileID)

#prorep_csv = io.BytesIO()
#downloader = MediaIoBaseDownload(prorep_csv, request)
#done = False
#while done is False:
#  status, done = downloader.next_chunk()
#prorep_csv.seek(0)
#load it into pandas
df = pd.read_csv(
'//content/pythonProject/sourceCSV/warehousedata-prodsheet-20220101.csv'
,encoding = "ISO-8859-1"
,usecols=[0,1,2,3,4,5,7,8,9,10,17,18,19,20,23,25,27,29,31,33,35,37,55,82]
,names = ['SystemID', 'Date', 'Associate','WorkCenter','Activity','Allocation','Units','ActualHours','EarnedHours','StopTime','AssociateName','WorkCenterDesc','ActivityDesc','AllocationDesc','UnitsE1','LinesE2','OrderE3','LocationsE4','LPNSE5','EachesE6','CasesE7','PalletsE8','TravelTimeE17','Supervisor']
,skip_blank_lines = True
,low_memory=False 
)
print(df)

# V V V V V CSV fields V V V V V
#SystemID	INTEGER
df['SystemID'] = (df['SystemID']).astype(int) 
#Date	DATE      ##convert Date from Serial Date to Python DateTime
df["Date"] = df["Date"].map(lambda x: dt.datetime.fromordinal(dt.datetime(1900, 1, 1).toordinal() + x - 2)).map(lambda x: x.strftime("%Y-%m-%d")).astype('datetime64[ns]')
#Associate	STRING
df['Associate'] = (df['Associate']).astype(str)
#WorkCenter	STRING
df['WorkCenter'] = (df['WorkCenter']).astype(str)
#Activity	STRING
df['Activity'] = (df['Activity']).astype(str)
#Allocation	INTEGER
df['Allocation'] = (df['Allocation']).astype(int)
#Units	INTEGER
df['Units'] = (df['Units']).astype(int)
#ActualHours	FLOAT
df['ActualHours'] = (df['ActualHours']).astype(float)
#EarnedHours	FLOAT
df['EarnedHours1'] = (df['EarnedHours']).astype(float)
#StopTime	TIME
df['StopTime'] = pd.to_datetime((df["StopTime"]*86400), unit='s').dt.strftime("%H:%M:%S")
#AssociateName	STRING
df['AssociateName'] = (df['AssociateName']).astype(str)
#WorkCenterDesc	STRING
df['WorkCenterDesc'] = (df['WorkCenterDesc']).astype(str)
#ActivityDesc	STRING
df['ActivityDesc'] = (df['ActivityDesc']).astype(str)
#AllocationDesc	STRING	NULLABLE	
df['AllocationDesc'] = (df['AllocationDesc']).astype(str)
#UnitsE1	INTEGER	
df['UnitsE1'] = (df['UnitsE1']).astype(float)#should be int
#LinesE2	INTEGER
df['LinesE2'] = (df['LinesE2']).astype(float)#should be int
#OrderE3	INTEGER
df['OrderE3'] = (df['OrderE3']).astype(float)#should be int
#LocationsE4	INTEGER
df['LocationsE4'] = (df['LocationsE4']).astype(float)#should be int
#LPNSE5	INTEGER
df['LPNSE5'] = (df['LPNSE5']).astype(float)#should be int
#EachesE6	INTEGER
df['EachesE6'] = (df['EachesE6']).astype(float)#should be int
#CasesE7	INTEGER
df['CasesE7'] = (df['CasesE7']).astype(float)#should be int
#PalletsE8	INTEGER
df['PalletsE8'] = (df['PalletsE8']).astype(float)#should be int
#TravelTimeE17	FLOAT
df['TravelTimeE17'] = (df['TravelTimeE17']).astype(float)
# V V V V V fields not in CSV from ProRep V V V V V
#ReportDateTime	DATETIME
df['ReportDateTime'] = (df['Date'].astype(str) + " " + df['StopTime'].astype(str)).astype('datetime64[ns]').astype('datetime64[ns]')
#Shift	INTEGER
df['Shift'] = df['WorkCenter'].str[-1:].astype(int)
#UnitsInTOW	INTEGER
#df.loc[df['UnitsCountsForDept'] == 'Yes', 'UnitsInTOW'] = df['Units'].astype(float)#should be int
#IndirectHours	FLOAT
#df.loc[df['Direct'] == 'Indirect', 'IndirectHours'] = df['ActualHours'].astype(float)
#DirectHours	FLOAT
#df.loc[df['Direct'] == 'Direct', 'DirectHours'] = df['ActualHours'].astype(float)
#GapHours	FLOAT
ifThese = [df['Activity'].str[0:4] == '9997'
    ,df['ActualHours'] == 0
    ,(((df['EarnedHours']/df['ActualHours']) < 0.3)*((df['EarnedHours']/df['ActualHours']) > 0)*(df['ActualHours'] >0.5)) > 0]
thenThese = [df['ActualHours']
    ,0
    ,df['ActualHours']-df['EarnedHours']]
df['GapHours'] = np.select(ifThese, thenThese,0)
#TimeStamp	DATETIME
df['TimeStamp'] = dt.datetime.now().astimezone(tz.timezone('US/Eastern')).strftime('%Y-%m-%dT%H:%M:%S')
df['TimeStamp'] = df['TimeStamp'].astype('datetime64[ns]')
#weekOf	DATE
df['weekOf'] = pd.to_datetime(df['Date']) - pd.offsets.Week(weekday=6)
#Supervisor	INTEGER
df['Supervisor'] = (df['Supervisor']).astype(float)#should be int

#Put columns in order
#df = df.reindex(columns=['SystemID', 'Date', 'Associate', 'WorkCenter', 'Activity', 'Allocation', 'Units', 'ActualHours', 'EarnedHours', 'StopTime', 'AssociateName', 'WorkCenterDesc', 'ActivityDesc', 'AllocationDesc', 'UnitsE1', 'LinesE2', 'OrderE3', 'LocationsE4', 'LPNSE5', 'EachesE6', 'CasesE7', 'PalletsE8', 'TravelTimeE17', 'ReportDateTime', 'Shift', 'TypeWork', 'UnitsCountsForDept', 'UnitsInTOW', 'Direct', 'IndirectHours', 'DirectHours', 'HomeWC', 'Dept', 'GapHours', 'PlanIndirectHours', 'ReportingArea', 'UnitsCounting', 'TimeStamp', 'whName', 'whRegion','srcGoal', 'functionGroup', 'functionSubGroup', 'Measured', 'dataSource', 'weekOf', 'ProductType', 'Supervisor', 'SupervisorName', 'srcGoalDept', 'gpOperation'])

#Display interactive table
#from google.colab.data_table import DataTable
#DataTable(df.head(100), include_index=False, num_rows_per_page=25, max_columns=60)

#print CSV for Data Validation vs GBQ table
exportCSV = df.to_csv('//content/pythonProject/bigQueryCSV/output.csv', index=False)

      SystemID   Date    Associate  WorkCenter  Activity  Allocation  Units  \
0           99  44562  AUTOPROCESS         161      1600           0    231   
1           99  44562  AUTOPROCESS         161      1600           0    474   
2           99  44562  AUTOPROCESS         161      1600           0    117   
3           99  44562  AUTOPROCESS         161      1600           0    141   
4           99  44562  AUTOPROCESS         162      1600           0    293   
...        ...    ...          ...         ...       ...         ...    ...   
3525        99  44562      4047454         161      1610           0      1   
3526        99  44562      4047454         161      5600           0      1   
3527        99  44562      4047454         161      1610           0      1   
3528        99  44562      4047454         161      5600           0      1   
3529        99  44562      4047454         161   9997161           0      0   

      ActualHours  EarnedHours  StopTime  ... Units